<a href="https://colab.research.google.com/github/Christi049/Smart-Pantry/blob/main/smart_pantry_finetunedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

file = json.load(open("Dataset.json", "r"))
print(file[1])

{'instruction': 'Suggest a weekly meal plan for 4 people, prioritizing ingredients that will expire soon. The meal plan must strictly adhere to the specified dietary restrictions and be free of all listed allergens. The plan should include three meals per day, with varied dishes, and minimal extra shopping. Leftovers should be used to reduce cooking effort and food waste.', 'input': {'serving_size': 4, 'dietary_restrictions': ['Vegetarian'], 'allergens': ['None'], 'ingredients': [{'name': 'Paneer', 'quantity': '500g', 'expiry': '2025-09-04'}, {'name': 'Bell peppers', 'quantity': '4 large', 'expiry': '2025-09-05'}, {'name': 'Onions', 'quantity': '6 medium', 'expiry': '2025-09-10'}, {'name': 'Tomatoes', 'quantity': '6 large', 'expiry': '2025-09-06'}, {'name': 'Yogurt', 'quantity': '300g', 'expiry': '2025-09-03'}, {'name': 'Milk', 'quantity': '1000ml', 'expiry': '2025-09-03'}]}, 'output': {'meals': {'Monday': {'breakfast': {'description_for_model': 'Lassi (Yogurt drink) using the full 300

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.

In [ ]:
!pip install -q huggingface_hub transformers gradio accelerate

In [ ]:
# Only if you want to include your existing examples
with open("Dataset.json", "r") as f:
    current_examples = json.load(f)

In [ ]:
import random
import json
import itertools

def generate_accurate_examples(num_examples):
    # A more extensive list of ingredients and dietary information
    all_ingredients = {
        "eggs": {"vegan": False, "dairy": False, "gluten": True},
        "cheese": {"vegan": False, "dairy": True, "gluten": True},
        "spinach": {"vegan": True, "dairy": False, "gluten": True},
        "tofu": {"vegan": True, "dairy": False, "gluten": True, "soy": True},
        "bread": {"vegan": True, "dairy": False, "gluten": False},
        "chicken": {"vegan": False, "dairy": False, "gluten": True},
        "lentils": {"vegan": True, "dairy": False, "gluten": False},
        "coconut milk": {"vegan": True, "dairy": False, "gluten": True},
        "rice": {"vegan": True, "dairy": False, "gluten": False},
        "potatoes": {"vegan": True, "dairy": False, "gluten": True},
        "onions": {"vegan": True, "dairy": False, "gluten": True},
        "bell peppers": {"vegan": True, "dairy": False, "gluten": True},
        "nuts": {"vegan": True, "dairy": False, "gluten": True, "nuts": True},
        "avocado": {"vegan": True, "dairy": False, "gluten": True}  # ADDED TO FIX KEYERROR
    }

    # A simple, rule-based recipe database for a single meal
    recipes = {
        "Breakfast": {
            "Omelette": ["eggs", "cheese", "spinach"],
            "Scrambled Tofu": ["tofu", "spinach", "onions"],
            "Toast with Avocado": ["bread", "avocado"]
        },
        "Lunch": {
            "Chicken and Veggies": ["chicken", "rice", "bell peppers"],
            "Lentil Soup": ["lentils", "onions", "carrots"],
            "Veggie Burger": ["lentils", "bread", "spinach"]
        },
        "Dinner": {
            "Chicken Curry": ["chicken", "coconut milk", "rice"],
            "Spinach and Potatoes": ["potatoes", "spinach", "onions"],
            "Vegan Stir-fry": ["tofu", "rice", "bell peppers"]
        }
    }

    dietary_options = ["None", "Vegetarian", "Vegan", "Gluten-Free"]
    allergen_options = ["None", "Nuts", "Dairy", "Soy", "Gluten"]
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    new_examples = []

    for _ in range(num_examples):
        # 1. Randomly generate input parameters for the example
        serving_size = random.randint(1, 4)
        dietary_restriction = random.choice(dietary_options)
        allergens = random.sample(allergen_options, k=random.randint(0, 2))
        available_ingredients_list = random.sample(list(all_ingredients.keys()), k=random.randint(3, 6))

        # 2. Dynamically generate the output (meal plan and shopping list)
        meal_plan = {}
        shopping_list = set()

        for day in days:
            meals = {}
            for meal_type, meal_recipes in recipes.items():
                valid_recipes = []
                for recipe_name, required_ingredients in meal_recipes.items():
                    is_compatible = True
                    for ingredient in required_ingredients:
                        # Check if ingredient is in our master list before trying to access it.
                        if ingredient not in all_ingredients:
                            is_compatible = False
                            break

                        # Check for general dietary restrictions
                        if dietary_restriction == "Vegetarian" and all_ingredients[ingredient]["vegan"] == False:
                            is_compatible = False
                            break
                        if dietary_restriction == "Vegan" and all_ingredients[ingredient]["vegan"] == False:
                            is_compatible = False
                            break

                        # Check for allergens by using .get() or 'in' keyword to avoid KeyError
                        for allergen in allergens:
                            if all_ingredients[ingredient].get(allergen, False) == True:
                                is_compatible = False
                                break

                        if not is_compatible:
                            break

                    if is_compatible:
                        valid_recipes.append((recipe_name, required_ingredients))

                if not valid_recipes:
                    meals[meal_type] = {"description_for_model": "Simple meal", "display_text": "Simple meal"}
                    continue

                chosen_recipe_name, chosen_ingredients = random.choice(valid_recipes)

                for ingredient in chosen_ingredients:
                    if ingredient not in available_ingredients_list:
                        shopping_list.add(ingredient)

                ingredient_str = ", ".join(chosen_ingredients)
                description = f"Prepare {chosen_recipe_name} with {ingredient_str}. This is a {dietary_restriction} and {', '.join(allergens)} free meal."
                meals[meal_type] = {
                    "description_for_model": description,
                    "display_text": chosen_recipe_name
                }
            meal_plan[day] = meals

        # 3. Assemble the final example
        instruction = (
            f"Suggest a weekly meal plan for {serving_size} person(s), prioritizing "
            "already available ingredients. The plan must adhere to the specified dietary "
            "restrictions and be free of all listed allergens. Also provide a shopping list for any items not present."
        )
        input_data = {
            "serving_size": serving_size,
            "dietary_restrictions": [dietary_restriction],
            "allergens": allergens,
            "ingredients": [{"name": i, "quantity": f"{random.randint(100, 500)}g", "expiry": "2025-09-03"} for i in available_ingredients_list]
        }
        output_data = {
            "meals": meal_plan,
            "shopping_list": list(shopping_list)
        }

        new_examples.append({
            "instruction": instruction,
            "input": input_data,
            "output": output_data
        })

    return new_examples
new_examples = generate_accurate_examples(320)

In [ ]:
# Only if you loaded current_examples
combined_dataset = current_examples + new_examples

In [ ]:
# Save new dataset only
with open("dataset.json", "w") as f:
    json.dump(new_examples, f, indent=2)

# Save combined dataset (if combined)
with open("combined_dataset.json", "w") as f:
    json.dump(combined_dataset, f, indent=2)


In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
import json
from datasets import Dataset

def format_prompt(example):
    return (
        f"### Instruction: {example['instruction']}\n"
        f"### Input: {json.dumps(example['input'], ensure_ascii=False)}\n"
        f"### Output: {json.dumps(example['output'], ensure_ascii=False)}<|endoftext|>"
    )

# Use your combined_dataset instead of 'file'
formatted_data = [format_prompt(item) for item in combined_dataset]

# Create a Hugging Face Dataset
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.9.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

In [ ]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 428 | Num Epochs = 3 | Total steps = 162
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 97,255,424 of 3,310,005,248 (2.94% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,0.291300
50,0.129000
75,0.109000
100,0.102200
125,0.083600
150,0.074600


In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt (dataset format)
test_instruction = "Suggest a weekly meal plan for 2 people, prioritizing ingredients that will expire soon. The meal plan must strictly adhere to the specified dietary restrictions and be free of all listed allergens. The plan should include three meals per day, with varied dishes, and minimal extra shopping. Leftovers should be used to reduce cooking effort and food waste. Also include popular Western dishes available in India and Kerala dishes."

test_input = {
    "serving_size": 2,
    "dietary_restrictions": [
        "Vegan"
    ],
    "allergens": [
        "Nuts"
    ],
    "ingredients": [
        {
            "name": "Lentils (Toor Dal)",
            "quantity": "250g",
            "expiry": "2025-09-02"
        },
        {
            "name": "Tofu",
            "quantity": "200g",
            "expiry": "2025-09-03"
        },
        {
            "name": "Spinach",
            "quantity": "1 bunch",
            "expiry": "2025-09-04"
        },
        {
            "name": "Cabbage",
            "quantity": "1 medium head",
            "expiry": "2025-09-07"
        },
        {
            "name": "Potatoes",
            "quantity": "4 medium",
            "expiry": "2025-09-10"
        },
        {
            "name": "Rice Flour",
            "quantity": "300g",
            "expiry": "2025-10-01"
        }
    ]
}

# The messages list must have a "content" key
# We will use the format you fine-tuned the model on.
messages = [
    {"role": "user", "content": f"### Instruction: {test_instruction}\n### Input: {json.dumps(test_input, ensure_ascii=False)}\n### Output:"}
]

# Step 1: Get chat-formatted text
chat_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,              # <-- don't tokenize yet
    add_generation_prompt=True
)

# Step 2: Tokenize with attention_mask
inputs = tokenizer(
    chat_text,
    return_tensors="pt",
    return_attention_mask=True, # <-- ensures attention_mask is included
).to("cuda")

# Step 3: Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=1024,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
response = response.split("<|endoftext|>")[0]
print(response)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 20 Sep 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

### Instruction: Suggest a weekly meal plan for 2 people, prioritizing ingredients that will expire soon. The meal plan must strictly adhere to the specified dietary restrictions and be free of all listed allergens. The plan should include three meals per day, with varied dishes, and minimal extra shopping. Leftovers should be used to reduce cooking effort and food waste. Also include popular Western dishes available in India and Kerala dishes.
### Input: {"serving_size": 2, "dietary_restrictions": ["Vegan"], "allergens": ["Nuts"], "ingredients": [{"name": "Lentils (Toor Dal)", "quantity": "250g", "expiry": "2025-09-02"}, {"name": "Tofu", "quantity": "200g", "expiry": "2025-09-03"}, {"name": "Spinach", "quantity": "1 bunch", "expiry": "2025-09-04"}, {"name": "Cabbage", "quantity": "1 medium head"

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
adapter_path = "outputs/checkpoint-168"   # replace with your latest checkpoint

# Save tokenizer (it’s from the base model)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained("meal-gen-adapter")

# Just copy adapter folder (LoRA) for upload
!cp -r {adapter_path} meal-gen-adapter/


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

cp: cannot stat 'outputs/checkpoint-168': No such file or directory


In [ ]:
!ls meal-gen-adapter


chat_template.jinja	 tokenizer_config.json
special_tokens_map.json  tokenizer.json


In [ ]:
from transformers import AutoTokenizer

base_model_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained("meal-gen-adapter")


('meal-gen-adapter/tokenizer_config.json',
 'meal-gen-adapter/special_tokens_map.json',
 'meal-gen-adapter/chat_template.jinja',
 'meal-gen-adapter/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from huggingface_hub import create_repo

# This will create "meal-gen-adapter" under your account Christi049
create_repo("meal-gen-adapter", exist_ok=True)


RepoUrl('https://huggingface.co/Christi049/meal-gen-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='Christi049/meal-gen-adapter')

In [ ]:
from huggingface_hub import upload_folder

repo_id = "Christi049/meal-gen-adapter"  # replace with your HF username

upload_folder(
    folder_path="meal-gen-adapter",
    repo_id=repo_id,
    repo_type="model"
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ent/meal-gen-adapter/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Christi049/meal-gen-adapter/commit/2c0e606ad4e9c02383770f1ff5e78e872837a913', commit_message='Upload folder using huggingface_hub', commit_description='', oid='2c0e606ad4e9c02383770f1ff5e78e872837a913', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Christi049/meal-gen-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='Christi049/meal-gen-adapter'), pr_revision=None, pr_num=None)

In [ ]:
# Folder to save full PyTorch model + tokenizer
save_folder = "full_model"

# Save model in PyTorch format
model.save_pretrained(save_folder)

# Save tokenizer
tokenizer.save_pretrained(save_folder)

# Zip the folder for download / IBM WML
import shutil
shutil.make_archive("full_model", 'zip', save_folder)

# Download the zip file
from google.colab import files
files.download("full_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.68 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 14.53it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving gguf_model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving gguf_model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at gguf_model into f16 GGUF format.
The output location will be /content/gguf_model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: gguf_model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:h

In [ ]:
!zip -r gguf_model.zip gguf_model
from google.colab import files
files.download("gguf_model.zip")


  adding: gguf_model/ (stored 0%)
  adding: gguf_model/tokenizer.json (deflated 85%)
  adding: gguf_model/tokenizer_config.json (deflated 96%)
  adding: gguf_model/chat_template.jinja (deflated 71%)
  adding: gguf_model/special_tokens_map.json (deflated 71%)
  adding: gguf_model/pytorch_model-00001-of-00002.bin (deflated 32%)
  adding: gguf_model/unsloth.F16.gguf (deflated 32%)
  adding: gguf_model/generation_config.json (deflated 38%)
  adding: gguf_model/unsloth.Q4_K_M.gguf (deflated 2%)
  adding: gguf_model/pytorch_model-00002-of-00002.bin (deflated 33%)
  adding: gguf_model/config.json (deflated 53%)
  adding: gguf_model/pytorch_model.bin.index.json (deflated 95%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/gguf_model/unsloth.Q4_K_M.gguf /content/drive/MyDrive/


Mounted at /content/drive
